In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_addons as tfa  # Import the tensorflow_addons module

# Set image size and number of classes
img_size = (244, 244)
num_classes = 12  # Replace with the actual number of classes in your dataset

# Load MobileNetV2 pre-trained on ImageNet data
base_model = tf.keras.applications.MobileNetV2(input_shape=(img_size[0], img_size[1], 3), include_top=False, weights='imagenet')

# Freeze the convolutional base
base_model.trainable = False

# Create your own model on top of MobileNetV2
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model using Rectified Adam (RAdam) optimizer from tensorflow_addons
radam_optimizer = tfa.optimizers.RectifiedAdam(learning_rate=1e-4)
model.compile(optimizer=radam_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 8, 8, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d_2  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_4 (Dense)             (None, 128)               163968    
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 12)                1548      
                                                                 
Total params: 2423500 (9.24 MB)
Trainable params: 1655

In [4]:
# Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1
)

# Assuming you have a directory structure with train and validation sets
train_generator = train_datagen.flow_from_directory(
    '/Users/kevinsam/Desktop/Unif/Master/ThesisWheelchair/DATA/RoadSaW/RoadSaW-075_s/train',
    target_size=img_size,
    batch_size=32,
    class_mode='categorical'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    '/Users/kevinsam/Desktop/Unif/Master/ThesisWheelchair/DATA/RoadSaW/RoadSaW-075_s/validation',
    target_size=img_size,
    batch_size=32,
    class_mode='categorical'
)

Found 15516 images belonging to 12 classes.
Found 4944 images belonging to 12 classes.


### Train the model in two steps as described
#### First step

In [5]:

hist1 = model.fit(train_generator, epochs=10, validation_data=validation_generator)





Epoch 1/10
485/485 [==============================] - 221s 451ms/step - loss: 2.2592 - accuracy: 0.2365 - val_loss: 1.7777 - val_accuracy: 0.4780
Epoch 2/10
485/485 [==============================] - 219s 452ms/step - loss: 1.5465 - accuracy: 0.4541 - val_loss: 1.5334 - val_accuracy: 0.5336
Epoch 3/10
485/485 [==============================] - 220s 453ms/step - loss: 1.3108 - accuracy: 0.5242 - val_loss: 1.4174 - val_accuracy: 0.5415
Epoch 4/10
485/485 [==============================] - 222s 458ms/step - loss: 1.1788 - accuracy: 0.5666 - val_loss: 1.3324 - val_accuracy: 0.5629
Epoch 5/10
485/485 [==============================] - 222s 457ms/step - loss: 1.0889 - accuracy: 0.5951 - val_loss: 1.2806 - val_accuracy: 0.5690
Epoch 6/10
485/485 [==============================] - 221s 455ms/step - loss: 1.0319 - accuracy: 0.6088 - val_loss: 1.2348 - val_accuracy: 0.5702
Epoch 7/10
485/485 [==============================] - 220s 454ms/step - loss: 0.9835 - accuracy: 0.6264 - val_loss: 1.2055 -

AttributeError: module 'tensorflow.keras.optimizers' has no attribute 'RAdam'

In [7]:
# Unfreeze the convolutional base for fine-tuning
base_model.trainable = True
radam_optimizer2 = tfa.optimizers.RectifiedAdam(learning_rate=1e-5)
# Lower the learning rate for fine-tuning
model.compile(optimizer=radam_optimizer2, loss='categorical_crossentropy', metrics=['accuracy'])

#### Second step


In [8]:
hist2 = model.fit(train_generator, epochs=10, validation_data=validation_generator)

Epoch 1/10
460/485 [===========================>..] - ETA: 24s - loss: 2.3134 - accuracy: 0.2896